<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Demo : Test Logic

Version 1.0 (18/6/2018)

This **notebook** can be used to test logic gates


## Import and connection

In [ ]:
# Import the main SLab module
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

In [ ]:
# Get the Vref value
vref = slab.gdata['vref']

Execute the following **code cell** if you want the plots to be interactive.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

## Common code

Execute the following **code** cell to load the common functions needed for this notebook

In [ ]:
# Convert from voltage to level
def v2l(value):
    if value>0.8*vref: return '1';
    if value<0.2*vref: return '0';
    if value>0.4*vref and value<0.6*vref: return 'z'
    raise slab.SlabEx("Invalid logic level")

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="6"> One Input Gate Tests </font> 
<BR>
</div>

Tests a one input gate
You can use this code to test inverters or buffers, for instance.

* Connect **input** to **DAC1**
* Connect **output** to **ADC1**
* Connect a $220k\Omega$ resistor between **output** and $V_{DD}$
* Connect a $220k\Omega$ resistor between **output** and $GND$

The two $220k\Omega$ enables us to detect the **High Impedance** state.

## Truth Table Test

Check the truth table of the gate

In [ ]:
# Response for two inputs
slab.setVoltage(1,0)
L0 = v2l(slab.readVoltage(1))
slab.setVoltage(1,vref)
L1 = v2l(slab.readVoltage(1))
table = L0 + L1

# Print Truth table
print('Truth Table')
print()
print(' I |','O')
print(' --+--')
print(' 0 |',L0)
print(' 1 |',L1)
print()
if table == '01': print('Identified as Buffer')
if table == '10': print('Identified as Inverter')
if table == '0z': print('Identified as Buffer Open Drain')
if table == 'z0': print('Identified as Inverter Open Drain')

## DC Curve

The DC curve test checks the static transfer characteristic  
The sweep is performed in both direction so you can check the gate **hysteresis**  
A **half Vdd** level corresponds to **Z** state

In [ ]:
# DC Tests
dataF = slab.dcSweep(1,0,3,0.1)
dataR = slab.dcSweep(1,3,0,-0.1)
slab.plotnn([dataF[0],dataR[0]],[dataF[1],dataR[1]]
            ,'DC Gate Test','Vi (V)','Vo (V)'
            ,['Forward','Reverse'])

# AC Test

The AC test checks the response vs time  
A 5kHz square wave is introduced on the input  
A **half Vdd** level corresponds to **Z** state

In [ ]:
# 10 point square wave at 5kHz
slab.waveSquare(0,3,10) 
slab.setWaveFrequency(5000)

# Measure and plot
slab.tranStore(100,1)
slab.wavePlot()

<a id='system'></a>

<div class="alert alert-block alert-info">
<BR>
<font size="6"> Several Input Gate Tests </font> 
<BR>
</div>

Tests a several input gate

* Connect each **input** fom 0 to the **DIO** line with the same number
* Connect **output** to **ADC1**
* Connect a $220k\Omega$ resistor between **output** and $V_{DD}$
* Connect a $220k\Omega$ resistor between **output** and $GND$

The two $220k\Omega$ enables us to detect the **High Impedance** state.

Before excuting the specific tests, change and execute the following **code cell** to set the number of inputs of the gate. The code also initializes the **DIO** lines.

In [ ]:
# Set number of inputs
N = 2

# Initialize DIOs
for i in range(0,N):
    slab.dioMode(i,'output')

# Print information
print('The test Gate has',N,'inputs, from 0 to',N-1)

## Truth Table Test

Check the truth table of the gate  
The table is optimized to show **don't care** 'x's on the inputs

In [ ]:
# Number of combinations
ncomb = 2**N

# Initialize results
resList = []

# Test all cases
for case in range(0,ncomb):
    thisCase = []
    for bit in range(0,N):
        if case & (1<<bit):
            slab.dioWrite(bit,1)
            thisCase.append('1')
        else:
            slab.dioWrite(bit,0)
            thisCase.append('0')
    res = v2l(slab.readVoltage(1))
    thisCase.append(res)
    resList.append(thisCase) 
    
# Optimize function
def optimizationPass():
    global resList    
    lenlist = len(resList)
    if lenlist<2: return
    for i in range(0,lenlist-1):
        for bit in range(0,N):
            case = resList[i].copy() 
            if case[bit] == '0':
                case[bit] = '1'
                for j in range(i+1,lenlist):
                    if case == resList[j]:
                        resList[i][bit] = 'x'
                        del resList[j]
                        return 1
            if case[bit] == '1':
                case[bit] = '0'
                for j in range(i+1,lenlist):
                    if case == resList[j]:
                        resList[i][bit] = 'x'
                        del resList[j]
                        return 1   
    return 0

# Perform optimization
res = 1
while res:
    res=optimizationPass()
                    
# Print Truth table
print('Truth Table')
print('Inputs ordered 0..',N-1,sep='')
print()
for case in resList:
    for i in range(0,N):
        print(case[i],end='')
    print(' : ',case[-1],sep='')    

## Document license

Copyright  ©  Vicente Jiménez (2018)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">